In [41]:
#working

import fitz
import tkinter as tk
from tkinter import filedialog, messagebox
import io

class PDFRedactor:
    def __init__(self):
        self.window = tk.Tk()
        self.window.title("PDF Redaction Tool")
        
        # Improved Layout
        self.window.geometry("400x300")
        self.label = tk.Label(self.window, text="PDF Redaction Tool", font=("Helvetica", 16))
        self.label.pack(pady=10)
        
        self.open_button = tk.Button(self.window, text="Open PDF", command=self.open_pdf)
        self.open_button.pack(pady=10)

        self.redact_button = tk.Button(self.window, text="Redact Selected Areas", command=self.redact)
        self.redact_button.pack(pady=10)

        self.page_control_frame = tk.Frame(self.window)
        self.page_control_frame.pack(pady=10)

        self.prev_button = tk.Button(self.page_control_frame, text="Previous Page", command=self.previous_page)
        self.prev_button.grid(row=0, column=0, padx=5)

        self.page_label = tk.Label(self.page_control_frame, text="Page 1")
        self.page_label.grid(row=0, column=1)

        self.next_button = tk.Button(self.page_control_frame, text="Next Page", command=self.next_page)
        self.next_button.grid(row=0, column=2, padx=5)
        
        self.selected_rectangles = []

        self.window.mainloop()

    def open_pdf(self):
        file_path = filedialog.askopenfilename(title="Select a PDF File", filetypes=[("PDF Files", "*.pdf")])
        if file_path:
            self.pdf_document = fitz.open(file_path)
            self.total_pages = self.pdf_document.page_count
            self.current_page = 0
            self.selected_rectangles = [[] for _ in range(self.total_pages)]  
            self.display_pdf()
            
    def display_pdf(self):
        self.page_image = self.pdf_document[self.current_page].get_pixmap()
        self.img_bytes = io.BytesIO(self.page_image.tobytes())  
        
        # Display PDF page
        if hasattr(self, 'pdf_window'):
            self.pdf_window.destroy()
        
        self.pdf_window = tk.Toplevel(self.window)
        self.pdf_window.title(f"PDF Viewer - Page {self.current_page + 1}")
        
        self.canvas = tk.Canvas(self.pdf_window, width=self.page_image.width, height=self.page_image.height)
        self.canvas.pack()
        
        self.photo = tk.PhotoImage(data=self.img_bytes.getvalue())
        self.canvas.create_image(0, 0, image=self.photo, anchor='nw')
        
        # Binding mouse events
        self.canvas.bind("<ButtonPress-1>", self.on_button_press)
        self.canvas.bind("<ButtonRelease-1>", self.on_button_release)
        
        self.page_label.config(text=f"Page {self.current_page + 1} / {self.total_pages}")

    def on_button_press(self, event):
        self.start_x = event.x
        self.start_y = event.y

    def on_button_release(self, event):
        end_x = event.x
        end_y = event.y
        self.selected_rectangles[self.current_page].append((self.start_x, self.start_y, end_x, end_y))
        self.canvas.create_rectangle(self.start_x, self.start_y, end_x, end_y, outline="red")

    def redact(self):
        output_pdf = "redacted_output.pdf"
        for i, page in enumerate(self.pdf_document):
            for rect in self.selected_rectangles[i]:
                self.redact_area(page, rect)
        self.pdf_document.save(output_pdf)
        messagebox.showinfo("Success", f"Redacted PDF saved as: {output_pdf}")

    def redact_area(self, page, rect):
        rect_fitz = fitz.Rect(rect)
        page.add_redact_annot(rect_fitz)
        page.apply_redactions()

    def next_page(self):
        if self.current_page < self.total_pages - 1:
            self.current_page += 1
            self.display_pdf()

    def previous_page(self):
        if self.current_page > 0:
            self.current_page -= 1
            self.display_pdf()

if __name__ == "__main__":
    PDFRedactor()

In [ ]:
#improvement 1

In [83]:
import fitz
import tkinter as tk
from tkinter import filedialog, messagebox, simpledialog
import io
import re

class PDFRedactor:
    def __init__(self):
        self.window = tk.Tk()
        self.window.title("PDF Redaction Tool")
        
        # Improved Layout
        self.window.geometry("400x400")
        self.label = tk.Label(self.window, text="PDF Redaction Tool", font=("Helvetica", 16))
        self.label.pack(pady=10)
        
        self.open_button = tk.Button(self.window, text="Open PDF", command=self.open_pdf)
        self.open_button.pack(pady=10)

        self.redact_button = tk.Button(self.window, text="Redact Selected Areas", command=self.redact)
        self.redact_button.pack(pady=10)

        self.auto_redact_button = tk.Button(self.window, text="Auto Redact Sensitive Info", command=self.auto_redact)
        self.auto_redact_button.pack(pady=10)

        self.page_control_frame = tk.Frame(self.window)
        self.page_control_frame.pack(pady=10)

        self.prev_button = tk.Button(self.page_control_frame, text="Previous Page", command=self.previous_page)
        self.prev_button.grid(row=0, column=0, padx=5)

        self.page_label = tk.Label(self.page_control_frame, text="Page 1")
        self.page_label.grid(row=0, column=1)

        self.next_button = tk.Button(self.page_control_frame, text="Next Page", command=self.next_page)
        self.next_button.grid(row=0, column=2, padx=5)

        self.jump_button = tk.Button(self.page_control_frame, text="Jump to Page", command=self.jump_to_page)
        self.jump_button.grid(row=1, column=1, pady=5)

        self.zoom_in_button = tk.Button(self.page_control_frame, text="Zoom In", command=self.zoom_in)
        self.zoom_in_button.grid(row=2, column=0, pady=5)

        self.zoom_out_button = tk.Button(self.page_control_frame, text="Zoom Out", command=self.zoom_out)
        self.zoom_out_button.grid(row=2, column=2, pady=5)

        self.selected_rectangles = []
        self.zoom_level = 1.0
        self.patterns = [
            r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b",  # for Email pattern
            r"\b\d{10}\b",   # for phone no. pattern
        ]

        self.window.mainloop()

    def open_pdf(self):
        file_path = filedialog.askopenfilename(title="Select a PDF File", filetypes=[("PDF Files", "*.pdf")])
        if file_path:
            self.pdf_document = fitz.open(file_path)
            self.total_pages = self.pdf_document.page_count
            self.current_page = 0
            self.selected_rectangles = [[] for _ in range(self.total_pages)]  # List for rectangles for each page
            self.display_pdf()
            
    def display_pdf(self):
        self.page_image = self.pdf_document[self.current_page].get_pixmap(matrix=fitz.Matrix(self.zoom_level, self.zoom_level))
        self.img_bytes = io.BytesIO(self.page_image.tobytes())  
        
        if hasattr(self, 'pdf_window'):
            self.pdf_window.destroy()
        
        self.pdf_window = tk.Toplevel(self.window)
        self.pdf_window.title(f"PDF Viewer - Page {self.current_page + 1}")
        
        self.canvas = tk.Canvas(self.pdf_window, width=self.page_image.width, height=self.page_image.height)
        self.canvas.pack()
        
        self.photo = tk.PhotoImage(data=self.img_bytes.getvalue())
        self.canvas.create_image(0, 0, image=self.photo, anchor='nw')
        
        self.canvas.bind("<ButtonPress-1>", self.on_button_press)
        self.canvas.bind("<ButtonRelease-1>", self.on_button_release)
        
        self.page_label.config(text=f"Page {self.current_page + 1} / {self.total_pages}")

    def on_button_press(self, event):
        self.start_x = event.x
        self.start_y = event.y

    def on_button_release(self, event):
        end_x = event.x
        end_y = event.y
        self.selected_rectangles[self.current_page].append((self.start_x, self.start_y, end_x, end_y))
        self.canvas.create_rectangle(self.start_x, self.start_y, end_x, end_y, outline="red")

    def redact(self):
        output_pdf = "redacted_output.pdf"
        for i, page in enumerate(self.pdf_document):
            for rect in self.selected_rectangles[i]:
                self.redact_area(page, rect)
        self.pdf_document.save(output_pdf)
        messagebox.showinfo("Success", f"Redacted PDF saved as: {output_pdf}")

    def redact_area(self, page, rect):
        rect_fitz = fitz.Rect(rect)
        page.add_redact_annot(rect_fitz)
        page.apply_redactions()

    def next_page(self):
        if self.current_page < self.total_pages - 1:
            self.current_page += 1
            self.display_pdf()

    def previous_page(self):
        if self.current_page > 0:
            self.current_page -= 1
            self.display_pdf()

    def jump_to_page(self):
        page_number = simpledialog.askinteger("Jump to Page", f"Enter page number (1-{self.total_pages}):", minvalue=1, maxvalue=self.total_pages)
        if page_number:
            self.current_page = page_number - 1
            self.display_pdf()
 
    def zoom_in(self):
        self.zoom_level += 0.2
        self.display_pdf()

    def zoom_out(self):
        self.zoom_level = max(1.0, self.zoom_level - 0.2)  
        self.display_pdf()

    def auto_redact(self):
        keywords = simpledialog.askstring("Keywords", "Enter custom keywords to redact (comma separated):")
        if keywords:
            self.patterns += [r'\b' + re.escape(word.strip()) + r'\b' for word in keywords.split(',')]
        
        for page_num in range(self.total_pages):
            page = self.pdf_document.load_page(page_num)
            page_text = page.get_text("text")
            for pattern in self.patterns:
                matches = re.finditer(pattern, page_text)
                for match in matches:
                    highlight_areas = page.search_for(match.group())
                    for area in highlight_areas:
                        self.selected_rectangles[page_num].append(area)
                        self.redact_area(page, area)
        
        output_pdf = "auto_redacted_output.pdf"
        self.pdf_document.save(output_pdf)
        messagebox.showinfo("Success", f"Automatically redacted PDF saved as: {output_pdf}")

if __name__ == "__main__":
    PDFRedactor()


In [27]:
#improvements 2  ..... acceptable at this moment

In [79]:
import fitz
import tkinter as tk
from tkinter import filedialog, messagebox, simpledialog
import io
import re

class PDFRedactor:
    def __init__(self):
        self.window = tk.Tk()
        self.window.title("PDF Redaction Tool")
        
        self.window.geometry("400x500")
        self.label = tk.Label(self.window, text="PDF Redaction Tool", font=("Helvetica", 16))
        self.label.pack(pady=10)
        
        self.open_button = tk.Button(self.window, text="Open PDF", command=self.open_pdf)
        self.open_button.pack(pady=10)

        self.redact_button = tk.Button(self.window, text="Redact Selected Areas", command=self.redact)
        self.redact_button.pack(pady=10)

        self.auto_redact_button = tk.Button(self.window, text="Auto Redact Sensitive Info", command=self.select_redaction_level)
        self.auto_redact_button.pack(pady=10)

        self.page_control_frame = tk.Frame(self.window)
        self.page_control_frame.pack(pady=10)

        self.prev_button = tk.Button(self.page_control_frame, text="Previous Page", command=self.previous_page)
        self.prev_button.grid(row=0, column=0, padx=5)

        self.page_label = tk.Label(self.page_control_frame, text="Page 1")
        self.page_label.grid(row=0, column=1)

        self.next_button = tk.Button(self.page_control_frame, text="Next Page", command=self.next_page)
        self.next_button.grid(row=0, column=2, padx=5)

        self.jump_button = tk.Button(self.page_control_frame, text="Jump to Page", command=self.jump_to_page)
        self.jump_button.grid(row=1, column=1, pady=5)

        self.zoom_in_button = tk.Button(self.page_control_frame, text="Zoom In", command=self.zoom_in)
        self.zoom_in_button.grid(row=2, column=0, pady=5)

        self.zoom_out_button = tk.Button(self.page_control_frame, text="Zoom Out", command=self.zoom_out)
        self.zoom_out_button.grid(row=2, column=2, pady=5)

        self.selected_rectangles = []
        self.zoom_level = 1.0
        
        self.patterns = {
            'email': r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b",  
            'phone': r"\b\d{10}\b", 
            'custom': [], 
            'name': r"\b[A-Z][a-z]* [A-Z][a-z]*\b",  
            'date': r"\b\d{2}/\d{2}/\d{4}\b",  
        }

        self.redaction_level = 'low' 

        self.window.mainloop()

    def open_pdf(self):
        file_path = filedialog.askopenfilename(title="Select a PDF File", filetypes=[("PDF Files", "*.pdf")])
        if file_path:
            self.pdf_document = fitz.open(file_path)
            self.total_pages = self.pdf_document.page_count
            self.current_page = 0
            self.selected_rectangles = [[] for _ in range(self.total_pages)]  
            self.display_pdf()
            
    def display_pdf(self):
        self.page_image = self.pdf_document[self.current_page].get_pixmap(matrix=fitz.Matrix(self.zoom_level, self.zoom_level))
        self.img_bytes = io.BytesIO(self.page_image.tobytes())  
        
        # Display PDF pagesss......
        if hasattr(self, 'pdf_window'):
            self.pdf_window.destroy()
        
        self.pdf_window = tk.Toplevel(self.window)
        self.pdf_window.title(f"PDF Viewer - Page {self.current_page + 1}")
        
        self.canvas = tk.Canvas(self.pdf_window, width=self.page_image.width, height=self.page_image.height)
        self.canvas.pack()
        
        self.photo = tk.PhotoImage(data=self.img_bytes.getvalue())
        self.canvas.create_image(0, 0, image=self.photo, anchor='nw')
        
        self.canvas.bind("<ButtonPress-1>", self.on_button_press)
        self.canvas.bind("<ButtonRelease-1>", self.on_button_release)
        
        self.page_label.config(text=f"Page {self.current_page + 1} / {self.total_pages}")

    def on_button_press(self, event):
        self.start_x = event.x
        self.start_y = event.y

    def on_button_release(self, event):
        end_x = event.x
        end_y = event.y
        self.selected_rectangles[self.current_page].append((self.start_x, self.start_y, end_x, end_y))
        self.canvas.create_rectangle(self.start_x, self.start_y, end_x, end_y, outline="green")

    def redact(self):
        output_pdf = "redacted_output.pdf"
        for i, page in enumerate(self.pdf_document):
            for rect in self.selected_rectangles[i]:
                self.redact_area(page, rect)
        self.pdf_document.save(output_pdf)
        messagebox.showinfo("Success", f"Redacted PDF saved as: {output_pdf}")

    def redact_area(self, page, rect):
        rect_fitz = fitz.Rect(rect)
        page.add_redact_annot(rect_fitz, fill=(0, 0, 0))
        page.apply_redactions()

    def next_page(self):
        if self.current_page < self.total_pages - 1:
            self.current_page += 1
            self.display_pdf()

    def previous_page(self):
        if self.current_page > 0:
            self.current_page -= 1
            self.display_pdf()

    def jump_to_page(self):
        page_number = simpledialog.askinteger("Jump to Page", f"Enter page number (1-{self.total_pages}):", minvalue=1, maxvalue=self.total_pages)
        if page_number:
            self.current_page = page_number - 1
            self.display_pdf()

    def zoom_in(self):
        self.zoom_level += 0.2
        self.display_pdf()

    def zoom_out(self):
        self.zoom_level = max(1.0, self.zoom_level - 0.2)  
        self.display_pdf()

    def select_redaction_level(self):
        self.redaction_level = simpledialog.askstring("Redaction Level", "Enter redaction level (high, medium, low):").lower()
        
        if self.redaction_level not in ['high', 'medium', 'low']:
            messagebox.showwarning("Invalid Input", "Please select a valid redaction level: high, medium, or low.")
        else:
            self.auto_redact()

    def auto_redact(self):
        if self.redaction_level in ['medium', 'high']:
            keywords = simpledialog.askstring("Keywords", "Enter custom keywords to redact (comma separated):")
            if keywords:
                self.patterns['custom'] = [r'\b' + re.escape(word.strip()) + r'\b' for word in keywords.split(',')]

        for page_num in range(self.total_pages):
            page = self.pdf_document.load_page(page_num)
            page_text = page.get_text("text")
            patterns_to_apply = []

            if self.redaction_level == 'low':
                patterns_to_apply = [self.patterns['email'], self.patterns['phone']]
            elif self.redaction_level == 'medium':
                patterns_to_apply = [self.patterns['email'], self.patterns['phone']] + self.patterns['custom']
            elif self.redaction_level == 'high':
                patterns_to_apply = list(self.patterns.values())  

            for pattern in patterns_to_apply:
                matches = re.finditer(pattern, page_text)
                for match in matches:
                    highlight_areas = page.search_for(match.group())
                    for area in highlight_areas:
                        self.selected_rectangles[page_num].append(area)
                        self.redact_area(page, area)
        
        output_pdf = f"auto_redacted_output_{self.redaction_level}.pdf"
        self.pdf_document.save(output_pdf)
        messagebox.showinfo("Success", f"Automatically redacted PDF saved as: {output_pdf}")

if __name__ == "__main__":
    PDFRedactor()


In [24]:
#improvement .... perfectly working

In [80]:
import fitz
import tkinter as tk
from tkinter import filedialog, messagebox, simpledialog
import io
import re

class PDFRedactor:
    def __init__(self):
        self.window = tk.Tk()
        self.window.title("PDF Redaction Tool")
        self.window.geometry("400x500")
        
        # Title
        self.label = tk.Label(self.window, text="PDF Redaction Tool", font=("Helvetica", 16))
        self.label.pack(pady=10)
        
        # Open PDF Button
        self.open_button = tk.Button(self.window, text="Open PDF", command=self.open_pdf)
        self.open_button.pack(pady=10)

        # Redaction Level Button
        self.redact_button = tk.Button(self.window, text="Redact Automatically", command=self.select_redaction_level)
        self.redact_button.pack(pady=10)

        # Frame for page navigation
        self.page_control_frame = tk.Frame(self.window)
        self.page_control_frame.pack(pady=10)

        self.prev_button = tk.Button(self.page_control_frame, text="Previous Page", command=self.previous_page)
        self.prev_button.grid(row=0, column=0, padx=5)

        self.page_label = tk.Label(self.page_control_frame, text="Page 1")
        self.page_label.grid(row=0, column=1)

        self.next_button = tk.Button(self.page_control_frame, text="Next Page", command=self.next_page)
        self.next_button.grid(row=0, column=2, padx=5)

        self.selected_rectangles = []
        self.zoom_level = 1.0

        # Define patterns for entities
        self.patterns = {
            'name': r"\b[A-Z][a-z]* [A-Z][a-z]*\b",  
            'registration': r"\b[A-Z0-9]{10,15}\b",  
            'mobile': r"\b\d{10}\b",  
            'aadhaar': r"\b\d{4} \d{4} \d{4}\b",  
            'address': r"\b\d+\s+\w+\s+\w+",  
            'dob': r"\b\d{2}/\d{2}/\d{4}\b",  
            'signature': r"\b(Signature|Signed by)\b",
            'account_number': r"\b\d{9,18}\b"
        }

        self.colors = {
            'name': (1, 0, 0),  # Red
            'registration': (0, 1, 0),  # Green
            'mobile': (0, 0, 1),  # Blue
            'aadhaar': (1, 1, 0),  # Yellow
            'address': (1, 0.5, 0),  # Orange
            'dob': (0, 1, 1),  # Cyan
            'signature': (1, 0, 1),  # Magenta
            'account_number': (0.5, 0, 0.5)  # Purple
        }

        self.redaction_level = 'low'

        self.window.mainloop()

    def open_pdf(self):
        file_path = filedialog.askopenfilename(title="Select a PDF File", filetypes=[("PDF Files", "*.pdf")])
        if file_path:
            self.pdf_document = fitz.open(file_path)
            self.total_pages = self.pdf_document.page_count
            self.current_page = 0
            self.selected_rectangles = [[] for _ in range(self.total_pages)]  
            self.display_pdf()

    def display_pdf(self):
        self.page_image = self.pdf_document[self.current_page].get_pixmap(matrix=fitz.Matrix(self.zoom_level, self.zoom_level))
        self.img_bytes = io.BytesIO(self.page_image.tobytes())  
        
        if hasattr(self, 'pdf_window'):
            self.pdf_window.destroy()
        
        self.pdf_window = tk.Toplevel(self.window)
        self.pdf_window.title(f"PDF Viewer - Page {self.current_page + 1}")
        
        self.canvas = tk.Canvas(self.pdf_window, width=self.page_image.width, height=self.page_image.height)
        self.canvas.pack()

        self.photo = tk.PhotoImage(data=self.img_bytes.getvalue())
        self.canvas.create_image(0, 0, image=self.photo, anchor='nw')

        self.page_label.config(text=f"Page {self.current_page + 1} / {self.total_pages}")

    def select_redaction_level(self):
        self.redaction_level = simpledialog.askstring("Redaction Level", "Enter redaction level (high, medium, low):").lower()
        
        if self.redaction_level not in ['high', 'medium', 'low']:
            messagebox.showwarning("Invalid Input", "Please select a valid redaction level: high, medium, or low.")
        else:
            self.auto_highlight_and_redact()

    def auto_highlight_and_redact(self):
        for page_num in range(self.total_pages):
            page = self.pdf_document.load_page(page_num)
            page_text = page.get_text("text")

            patterns_to_apply = []

            if self.redaction_level == 'low':
                patterns_to_apply = ['mobile', 'account_number']
            elif self.redaction_level == 'medium':
                patterns_to_apply = ['mobile', 'account_number', 'aadhaar', 'name', 'dob']
            elif self.redaction_level == 'high':
                patterns_to_apply = list(self.patterns.keys())

            for entity in patterns_to_apply:
                matches = re.finditer(self.patterns[entity], page_text)
                for match in matches:
                    highlight_areas = page.search_for(match.group())
                    color = self.colors[entity]
                    for area in highlight_areas:
                        page.add_freetext_annot(area, "", text_color=color, fill_color=color)
                        self.redact_area(page, area)

        output_pdf = f"auto_redacted_output_{self.redaction_level}.pdf"
        self.pdf_document.save(output_pdf)
        messagebox.showinfo("Success", f"Automatically redacted PDF saved as: {output_pdf}")

    def redact_area(self, page, rect):
        rect_fitz = fitz.Rect(rect)
        page.add_redact_annot(rect_fitz, fill=(0, 0, 0))
        page.apply_redactions()

    def next_page(self):
        if self.current_page < self.total_pages - 1:
            self.current_page += 1
            self.display_pdf()

    def previous_page(self):
        if self.current_page > 0:
            self.current_page -= 1
            self.display_pdf()

if __name__ == "__main__":
    PDFRedactor()


In [85]:
#improvement... 04/10/24

In [ ]:
import fitz
import tkinter as tk
from tkinter import filedialog, messagebox, simpledialog
import io
import re

class PDFRedactor:
    def __init__(self):
        self.window = tk.Tk()
        self.window.title("PDF Redaction Tool")
        self.window.geometry("400x500")
        
        # Title
        self.label = tk.Label(self.window, text="PDF Redaction Tool", font=("Helvetica", 16))
        self.label.pack(pady=10)
        
        # Open PDF Button
        self.open_button = tk.Button(self.window, text="Open PDF", command=self.open_pdf)
        self.open_button.pack(pady=10)

        # Redaction Level Button
        self.redact_button = tk.Button(self.window, text="Redact Automatically", command=self.select_redaction_level)
        self.redact_button.pack(pady=10)

        # Manual Redaction Button
        self.manual_redact_button = tk.Button(self.window, text="Redact Manually", command=self.enable_manual_redaction)
        self.manual_redact_button.pack(pady=10)

        self.page_control_frame = tk.Frame(self.window)
        self.page_control_frame.pack(pady=10)

        self.prev_button = tk.Button(self.page_control_frame, text="Previous Page", command=self.previous_page)
        self.prev_button.grid(row=0, column=0, padx=5)

        self.page_label = tk.Label(self.page_control_frame, text="Page 1")
        self.page_label.grid(row=0, column=1)

        self.next_button = tk.Button(self.page_control_frame, text="Next Page", command=self.next_page)
        self.next_button.grid(row=0, column=2, padx=5)

        self.selected_rectangles = []
        self.manual_selection_enabled = False  # Flag for manual redaction
        self.zoom_level = 1.0

        self.patterns = {
            'name': r"\b[A-Z][a-z]* [A-Z][a-z]*\b",  
            'registration': r"\b[A-Z0-9]{10,15}\b",  
            'mobile': r"\b\d{10}\b",  
            'aadhaar': r"\b\d{4} \d{4} \d{4}\b",  
            'address': r"\b\d+\s+\w+\s+\w+",  
            'dob': r"\b\d{2}/\d{2}/\d{4}\b",  
            'signature': r"\b(Signature|Signed by)\b",
            'account_number': r"\b\d{9,18}\b"
        }

        self.colors = {
            'name': (1, 0, 0),  # Red
            'registration': (0, 1, 0),  # Green
            'mobile': (0, 0, 1),  # Blue
            'aadhaar': (1, 1, 0),  # Yellow
            'address': (1, 0.5, 0),  # Orange
            'dob': (0, 1, 1),  # Cyan
            'signature': (1, 0, 1),  # Magenta
            'account_number': (0.5, 0, 0.5)  # Purple
        }

        self.redaction_level = 'low'

        self.window.mainloop()

    def open_pdf(self):
        file_path = filedialog.askopenfilename(title="Select a PDF File", filetypes=[("PDF Files", "*.pdf")])
        if file_path:
            self.pdf_document = fitz.open(file_path)
            self.total_pages = self.pdf_document.page_count
            self.current_page = 0
            self.selected_rectangles = [[] for _ in range(self.total_pages)]  
            self.display_pdf()

    def display_pdf(self):
        self.page_image = self.pdf_document[self.current_page].get_pixmap(matrix=fitz.Matrix(self.zoom_level, self.zoom_level))
        self.img_bytes = io.BytesIO(self.page_image.tobytes())  
        
        if hasattr(self, 'pdf_window'):
            self.pdf_window.destroy()
        
        self.pdf_window = tk.Toplevel(self.window)
        self.pdf_window.title(f"PDF Viewer - Page {self.current_page + 1}")
        
        self.canvas = tk.Canvas(self.pdf_window, width=self.page_image.width, height=self.page_image.height)
        self.canvas.pack()

        self.photo = tk.PhotoImage(data=self.img_bytes.getvalue())
        self.canvas.create_image(0, 0, image=self.photo, anchor='nw')

        self.page_label.config(text=f"Page {self.current_page + 1} / {self.total_pages}")

        # Enable manual rectangle selection on canvas
        self.canvas.bind("<ButtonPress-1>", self.start_rectangle)
        self.canvas.bind("<B1-Motion>", self.update_rectangle)
        self.canvas.bind("<ButtonRelease-1>", self.end_rectangle)

    def select_redaction_level(self):
        self.redaction_level = simpledialog.askstring("Redaction Level", "Enter redaction level (high, medium, low):").lower()
        
        if self.redaction_level not in ['high', 'medium', 'low']:
            messagebox.showwarning("Invalid Input", "Please select a valid redaction level: high, medium, or low.")
        else:
            self.auto_highlight_and_redact()

    def auto_highlight_and_redact(self):
        for page_num in range(self.total_pages):
            page = self.pdf_document.load_page(page_num)
            page_text = page.get_text("text")

            patterns_to_apply = []

            if self.redaction_level == 'low':
                patterns_to_apply = ['mobile', 'account_number']
            elif self.redaction_level == 'medium':
                patterns_to_apply = ['mobile', 'account_number', 'aadhaar', 'name', 'dob']
            elif self.redaction_level == 'high':
                patterns_to_apply = list(self.patterns.keys())

            for entity in patterns_to_apply:
                matches = re.finditer(self.patterns[entity], page_text)
                for match in matches:
                    highlight_areas = page.search_for(match.group())
                    color = self.colors[entity]
                    for area in highlight_areas:
                        page.add_freetext_annot(area, "", text_color=color, fill_color=color)
                        self.redact_area(page, area)

        output_pdf = f"auto_redacted_output_{self.redaction_level}.pdf"
        self.pdf_document.save(output_pdf)
        messagebox.showinfo("Success", f"Automatically redacted PDF saved as: {output_pdf}")

    def redact_area(self, page, rect):
        rect_fitz = fitz.Rect(rect)
        page.add_redact_annot(rect_fitz, fill=(0, 0, 0))
        page.apply_redactions()

    def enable_manual_redaction(self):
        self.manual_selection_enabled = True
        messagebox.showinfo("Manual Redaction", "Click and drag on the PDF to select areas for manual redaction.")

    def start_rectangle(self, event):
        if self.manual_selection_enabled:
            self.rect_start = (event.x, event.y)
            self.rect = self.canvas.create_rectangle(self.rect_start[0], self.rect_start[1], event.x, event.y, outline="red")

    def update_rectangle(self, event):
        if self.manual_selection_enabled and self.rect:
            self.canvas.coords(self.rect, self.rect_start[0], self.rect_start[1], event.x, event.y)

    def end_rectangle(self, event):
        if self.manual_selection_enabled and self.rect:
            self.selected_rectangles[self.current_page].append(self.canvas.coords(self.rect))
            self.canvas.delete(self.rect)
            messagebox.showinfo("Manual Redaction", "Area selected for redaction.")
            self.manual_selection_enabled = False 

    def next_page(self):
        if self.current_page < self.total_pages - 1:
            self.current_page += 1
            self.display_pdf()

    def previous_page(self):
        if self.current_page > 0:
            self.current_page -= 1
            self.display_pdf()

if __name__ == "__main__":
    PDFRedactor()


In [ ]:
# heart touching improvement.... redact manual + automatically in one prg.

In [2]:
import fitz
import tkinter as tk
from tkinter import filedialog, messagebox, simpledialog
import io
import re

class PDFRedactor:
    def __init__(self):
        self.window = tk.Tk()
        self.window.title("PDF Redaction Tool")
        self.window.geometry("400x500")
        
        # Title
        self.label = tk.Label(self.window, text="PDF Redaction Tool", font=("Helvetica", 18))
        self.label.pack(pady=10)
        
        self.open_button = tk.Button(self.window, text="Open PDF", command=self.open_pdf)
        self.open_button.pack(pady=10)

        self.redact_button = tk.Button(self.window, text="Select Redaction Level", command=self.select_redaction_level)
        self.redact_button.pack(pady=10)

        self.page_control_frame = tk.Frame(self.window)
        self.page_control_frame.pack(pady=10)

        self.prev_button = tk.Button(self.page_control_frame, text="Previous Page", command=self.previous_page)
        self.prev_button.grid(row=0, column=0, padx=5)

        self.page_label = tk.Label(self.page_control_frame, text="Page 1")
        self.page_label.grid(row=0, column=1)

        self.next_button = tk.Button(self.page_control_frame, text="Next Page", command=self.next_page)
        self.next_button.grid(row=0, column=2, padx=5)

        self.selected_rectangles = []
        self.manual_selection_enabled = True  # Flag for manual redaction
        self.zoom_level = 1.0

        # Define patterns for entities
        self.patterns = {
            'name': r"\b[A-Z][a-z]* [A-Z][a-z]*\b",  
            'registration': r"\b[A-Z0-9]{10,15}\b",  
            'mobile': r"\b\d{10}\b",  
            'aadhaar': r"\b\d{4} \d{4} \d{4}\b",  
            'address': r"\b\d+\s+\w+\s+\w+",  
            'dob': r"\b\d{2}/\d{2}/\d{4}\b",  
            'signature': r"\b(Signature|Signed by)\b",
            'account_number': r"\b\d{9,18}\b"
        }

        self.colors = {
            'name': (1, 0, 0),  # Red
            'registration': (0, 1, 0),  # Green
            'mobile': (0, 0, 1),  # Blue
            'aadhaar': (1, 1, 0),  # Yellow
            'address': (1, 0.5, 0),  # Orange
            'dob': (0, 1, 1),  # Cyan
            'signature': (1, 0, 1),  # Magenta
            'account_number': (0.5, 0, 0.5)  # Purple
        }

        self.redaction_level = 'low'
        self.pdf_document = None
        self.current_page = 0
        self.total_pages = 0

        self.window.mainloop()

    def open_pdf(self):
        file_path = filedialog.askopenfilename(title="Select a PDF File", filetypes=[("PDF Files", "*.pdf")])
        if file_path:
            self.pdf_document = fitz.open(file_path)
            self.total_pages = self.pdf_document.page_count
            self.current_page = 0
            self.selected_rectangles = [[] for _ in range(self.total_pages)]  # Store manual selections for each page
            self.display_pdf()

    def display_pdf(self):
        if not self.pdf_document:
            return

        self.page_image = self.pdf_document[self.current_page].get_pixmap(matrix=fitz.Matrix(self.zoom_level, self.zoom_level))
        self.img_bytes = io.BytesIO(self.page_image.tobytes())  
        
        if hasattr(self, 'pdf_window'):
            self.pdf_window.destroy()
        
        self.pdf_window = tk.Toplevel(self.window)
        self.pdf_window.title(f"PDF Viewer - Page {self.current_page + 1}")
        
        self.canvas = tk.Canvas(self.pdf_window, width=self.page_image.width, height=self.page_image.height)
        self.canvas.pack()

        self.photo = tk.PhotoImage(data=self.img_bytes.getvalue())
        self.canvas.create_image(0, 0, image=self.photo, anchor='nw')

        self.page_label.config(text=f"Page {self.current_page + 1} / {self.total_pages}")

        # Enable manual rectangle selection on canvas
        self.canvas.bind("<ButtonPress-1>", self.start_rectangle)
        self.canvas.bind("<B1-Motion>", self.update_rectangle)
        self.canvas.bind("<ButtonRelease-1>", self.end_rectangle)

    def select_redaction_level(self):
        self.redaction_level = simpledialog.askstring("Redaction Level", "Enter redaction level (high, medium, low):").lower()
        
        if self.redaction_level not in ['high', 'medium', 'low']:
            messagebox.showwarning("Invalid Input", "Please select a valid redaction level: high, medium, or low.")
        else:
            self.auto_redact()

    def auto_redact(self):
        for page_num in range(self.total_pages):
            page = self.pdf_document.load_page(page_num)
            page_text = page.get_text("text")

            patterns_to_apply = []

            if self.redaction_level == 'low':
                patterns_to_apply = ['mobile', 'account_number']
            elif self.redaction_level == 'medium':
                patterns_to_apply = ['mobile', 'account_number', 'aadhaar', 'name', 'dob']
            elif self.redaction_level == 'high':
                patterns_to_apply = list(self.patterns.keys())

            # Highlight and redact based on redaction level
            for entity in patterns_to_apply:
                matches = re.finditer(self.patterns[entity], page_text)
                for match in matches:
                    highlight_areas = page.search_for(match.group())
                    for area in highlight_areas:
                        page.add_redact_annot(fitz.Rect(area), fill=(0, 0, 0))
                        page.apply_redactions()

            # Apply manual selections for redaction
            for rect_coords in self.selected_rectangles[page_num]:
                x0, y0, x1, y1 = rect_coords
                manual_rect = fitz.Rect(x0, y0, x1, y1)
                page.add_redact_annot(manual_rect, fill=(0, 0, 0))
                page.apply_redactions()

        output_pdf = f"redacted_output_{self.redaction_level}.pdf"
        self.pdf_document.save(output_pdf)
        messagebox.showinfo("Success", f"Redacted PDF saved as: {output_pdf}")

    def start_rectangle(self, event):
        if self.manual_selection_enabled:
            self.rect_start = (event.x, event.y)
            self.rect = self.canvas.create_rectangle(self.rect_start[0], self.rect_start[1], event.x, event.y, outline="red")

    def update_rectangle(self, event):
        if self.manual_selection_enabled and self.rect:
            self.canvas.coords(self.rect, self.rect_start[0], self.rect_start[1], event.x, event.y)

    def end_rectangle(self, event):
        if self.manual_selection_enabled and self.rect:
            self.selected_rectangles[self.current_page].append(self.canvas.coords(self.rect))
            self.canvas.delete(self.rect)

    def next_page(self):
        if self.current_page < self.total_pages - 1:
            self.current_page += 1
            self.display_pdf()

    def previous_page(self):
        if self.current_page > 0:
            self.current_page -= 1
            self.display_pdf()

if __name__ == "__main__":
    PDFRedactor()


In [114]:
#improvement ++

In [11]:
import fitz
import tkinter as tk
from tkinter import filedialog, messagebox, simpledialog
import io
import re

class PDFRedactor:
    def __init__(self):
        self.window = tk.Tk()
        self.window.title("PDF Redaction Tool")
        self.window.geometry("400x500")

        self.label = tk.Label(self.window, text="PDF Redaction Tool", font=("Helvetica", 18))
        self.label.pack(pady=10)

        self.open_button = tk.Button(self.window, text="Open PDF", command=self.open_pdf)
        self.open_button.pack(pady=10)

        self.redact_button = tk.Button(self.window, text="Select Redaction Level", command=self.select_redaction_level)
        self.redact_button.pack(pady=10)

        self.page_control_frame = tk.Frame(self.window)
        self.page_control_frame.pack(pady=10)

        self.prev_button = tk.Button(self.page_control_frame, text="Previous Page", command=self.previous_page)
        self.prev_button.grid(row=0, column=0, padx=5)

        self.page_label = tk.Label(self.page_control_frame, text="Page 1")
        self.page_label.grid(row=0, column=1)

        self.next_button = tk.Button(self.page_control_frame, text="Next Page", command=self.next_page)
        self.next_button.grid(row=0, column=2, padx=5)

        self.jump_button = tk.Button(self.page_control_frame, text="Jump to Page", command=self.jump_to_page)
        self.jump_button.grid(row=1, column=1, pady=5)

        self.selected_rectangles = []
        self.manual_selection_enabled = True
        self.zoom_level = 1.0

        # Define patterns for entities
        self.patterns = {
            'name': r"\b[A-Z][a-z]* [A-Z][a-z]*\b",
            'registration': r"\b[A-Z0-9]{10,15}\b",
            'mobile': r"\b\d{10}\b",
            'aadhaar': r"\b\d{4} \d{4} \d{4}\b",
            'address': r"\b\d+\s+\w+\s+\w+",
            'dob': r"\b\d{2}/\d{2}/\d{4}\b",
            'signature': r"\b(Signature|Signed by)\b",
            'account_number': r"\b\d{9,18}\b",
            'hindi_text': r"[\u0900-\u097F]+"  # Pattern for Hindi characters
        }

        self.colors = {
            'name': (1, 0, 0),
            'registration': (0, 1, 0),
            'mobile': (0, 0, 1),
            'aadhaar': (1, 1, 0),
            'address': (1, 0.5, 0),
            'dob': (0, 1, 1),
            'signature': (1, 0, 1),
            'account_number': (0.5, 0, 0.5),
            'hindi_text': (1, 0.65, 0)  # Color for Hindi text
        }

        self.redaction_level = 'low'
        self.pdf_document = None
        self.current_page = 0
        self.total_pages = 0

        self.window.mainloop()

    def open_pdf(self):
        file_path = filedialog.askopenfilename(title="Select a PDF File", filetypes=[("PDF Files", "*.pdf")])
        if file_path:
            self.pdf_document = fitz.open(file_path)
            self.total_pages = self.pdf_document.page_count
            self.current_page = 0
            self.selected_rectangles = [[] for _ in range(self.total_pages)]
            self.display_pdf()

    def display_pdf(self):
        if not self.pdf_document:
            return

        self.page_image = self.pdf_document[self.current_page].get_pixmap(matrix=fitz.Matrix(self.zoom_level, self.zoom_level))
        self.img_bytes = io.BytesIO(self.page_image.tobytes())

        if hasattr(self, 'pdf_window'):
            self.pdf_window.destroy()

        self.pdf_window = tk.Toplevel(self.window)
        self.pdf_window.title(f"PDF Viewer - Page {self.current_page + 1}")

        self.canvas = tk.Canvas(self.pdf_window, width=self.page_image.width, height=self.page_image.height)
        self.canvas.pack()

        self.photo = tk.PhotoImage(data=self.img_bytes.getvalue())
        self.canvas.create_image(0, 0, image=self.photo, anchor='nw')

        self.page_label.config(text=f"Page {self.current_page + 1} / {self.total_pages}")

        self.canvas.bind("<ButtonPress-1>", self.start_rectangle)
        self.canvas.bind("<B1-Motion>", self.update_rectangle)
        self.canvas.bind("<ButtonRelease-1>", self.end_rectangle)

    def select_redaction_level(self):
        self.redaction_level = simpledialog.askstring("Redaction Level", "Enter redaction level (high, medium, low):").lower()

        if self.redaction_level not in ['high', 'medium', 'low']:
            messagebox.showwarning("Invalid Input", "Please select a valid redaction level: high, medium, or low.")
        else:
            self.auto_redact()

    def auto_redact(self):
        for page_num in range(self.total_pages):
            page = self.pdf_document.load_page(page_num)
            page_text = page.get_text("text")

            patterns_to_apply = []

            if self.redaction_level == 'low':
                patterns_to_apply = ['mobile', 'account_number', 'hindi_text']
            elif self.redaction_level == 'medium':
                patterns_to_apply = ['mobile', 'account_number', 'aadhaar', 'name', 'dob', 'hindi_text']
            elif self.redaction_level == 'high':
                patterns_to_apply = list(self.patterns.keys())

            for entity in patterns_to_apply:
                matches = re.finditer(self.patterns[entity], page_text)
                for match in matches:
                    highlight_areas = page.search_for(match.group())
                    for area in highlight_areas:
                        page.add_redact_annot(area, fill=(0, 0, 0))
                        page.apply_redactions()

            for rect_coords in self.selected_rectangles[page_num]:
                x0, y0, x1, y1 = rect_coords
                manual_rect = fitz.Rect(x0, y0, x1, y1)
                page.add_redact_annot(manual_rect, fill=(0, 0, 0))
                page.apply_redactions()

        output_pdf = f"redacted_output_{self.redaction_level}.pdf"
        self.pdf_document.save(output_pdf)
        messagebox.showinfo("Success", f"Redacted PDF saved as: {output_pdf}")

    def start_rectangle(self, event):
        if self.manual_selection_enabled:
            self.rect_start = (event.x, event.y)
            self.rect = self.canvas.create_rectangle(self.rect_start[0], self.rect_start[1], event.x, event.y, outline="green")

    def update_rectangle(self, event):
        if self.manual_selection_enabled and self.rect:
            self.canvas.coords(self.rect, self.rect_start[0], self.rect_start[1], event.x, event.y)

    def end_rectangle(self, event):
        if self.manual_selection_enabled and self.rect:
            self.selected_rectangles[self.current_page].append(self.canvas.coords(self.rect))
            self.canvas.delete(self.rect)

    def next_page(self):
        if self.current_page < self.total_pages - 1:
            self.current_page += 1
            self.display_pdf()

    def previous_page(self):
        if self.current_page > 0:
            self.current_page -= 1
            self.display_pdf()

    def jump_to_page(self):
        page_num = simpledialog.askinteger("Jump to Page", "Enter page number:")
        if page_num is not None and 1 <= page_num <= self.total_pages:
            self.current_page = page_num - 1
            self.display_pdf()
        else:
            messagebox.showwarning("Invalid Page", f"Please enter a valid page number between 1 and {self.total_pages}.")

if __name__ == "__main__":
    PDFRedactor()


In [16]:
#passwd set (optional) + improvement ++

In [7]:
import fitz
import tkinter as tk
from tkinter import filedialog, messagebox, simpledialog
import io
import re
from PyPDF2 import PdfReader, PdfWriter

class PDFRedactor:
    def __init__(self):
        self.window = tk.Tk()
        self.window.title("PDF Redaction Tool")
        self.window.geometry("400x500")

        self.label = tk.Label(self.window, text="PDF Redaction Tool", font=("Helvetica", 18))
        self.label.pack(pady=10)

        self.open_button = tk.Button(self.window, text="Open PDF", command=self.open_pdf)
        self.open_button.pack(pady=10)

        self.redact_button = tk.Button(self.window, text="Select Redaction Level", command=self.select_redaction_level)
        self.redact_button.pack(pady=10)

        self.page_control_frame = tk.Frame(self.window)
        self.page_control_frame.pack(pady=10)

        self.prev_button = tk.Button(self.page_control_frame, text="Previous Page", command=self.previous_page)
        self.prev_button.grid(row=0, column=0, padx=5)

        self.page_label = tk.Label(self.page_control_frame, text="Page 1")
        self.page_label.grid(row=0, column=1)

        self.next_button = tk.Button(self.page_control_frame, text="Next Page", command=self.next_page)
        self.next_button.grid(row=0, column=2, padx=5)

        self.jump_button = tk.Button(self.window, text="Jump to Page", command=self.jump_to_page)
        self.jump_button.pack(pady=10)

        self.selected_rectangles = []
        self.manual_selection_enabled = True
        self.zoom_level = 1.0

        self.patterns = {
            'name': r"\b[A-Z][a-z]* [A-Z][a-z]*\b",
            'registration': r"\b[A-Z0-9]{10,15}\b",
            'mobile': r"\b\d{10}\b",
            'aadhaar': r"\b\d{4} \d{4} \d{4}\b",
            'address': r"\b\d+\s+\w+\s+\w+",
            'dob': r"\b\d{2}/\d{2}/\d{4}\b",
            'signature': r"\b(Signature|Signed by)\b",
            'account_number': r"\b\d{9,18}\b",
            'hindi_text': r"[\u0900-\u097F]+"
        }

        self.colors = {
            'name': (1, 0, 0),
            'registration': (0, 1, 0),
            'mobile': (0, 0, 1),
            'aadhaar': (1, 1, 0),
            'address': (1, 0.5, 0),
            'dob': (0, 1, 1),
            'signature': (1, 0, 1),
            'account_number': (0.5, 0, 0.5),
            'hindi_text': (1, 0.65, 0)  # Color for Hindi text
        }

        self.redaction_level = 'low'
        self.pdf_document = None
        self.current_page = 0
        self.total_pages = 0
        self.window.mainloop()

    def open_pdf(self):
        file_path = filedialog.askopenfilename(title="Select a PDF File", filetypes=[("PDF Files", "*.pdf")])
        if file_path:
            self.pdf_document = fitz.open(file_path)
            self.total_pages = self.pdf_document.page_count
            self.current_page = 0
            self.selected_rectangles = [[] for _ in range(self.total_pages)]
            self.display_pdf()

    def display_pdf(self):
        if not self.pdf_document:
            return

        self.page_image = self.pdf_document[self.current_page].get_pixmap(matrix=fitz.Matrix(self.zoom_level, self.zoom_level))
        self.img_bytes = io.BytesIO(self.page_image.tobytes())

        if hasattr(self, 'pdf_window'):
            self.pdf_window.destroy()

        self.pdf_window = tk.Toplevel(self.window)
        self.pdf_window.title(f"PDF Viewer - Page {self.current_page + 1}")

        self.canvas = tk.Canvas(self.pdf_window, width=self.page_image.width, height=self.page_image.height)
        self.canvas.pack()

        self.photo = tk.PhotoImage(data=self.img_bytes.getvalue())
        self.canvas.create_image(0, 0, image=self.photo, anchor='nw')

        self.page_label.config(text=f"Page {self.current_page + 1} / {self.total_pages}")

        self.canvas.bind("<ButtonPress-1>", self.start_rectangle)
        self.canvas.bind("<B1-Motion>", self.update_rectangle)
        self.canvas.bind("<ButtonRelease-1>", self.end_rectangle)

    def select_redaction_level(self):
        self.redaction_level = simpledialog.askstring("Redaction Level", "Enter redaction level (high, medium, low):").lower()

        if self.redaction_level not in ['high', 'medium', 'low']:
            messagebox.showwarning("Invalid Input", "Please select a valid redaction level: high, medium, or low.")
        else:
            self.auto_redact()

    def auto_redact(self):
        for page_num in range(self.total_pages):
            page = self.pdf_document.load_page(page_num)
            page_text = page.get_text("text")

            patterns_to_apply = []

            if self.redaction_level == 'low':
                patterns_to_apply = ['mobile', 'account_number', 'hindi_text']
            elif self.redaction_level == 'medium':
                patterns_to_apply = ['mobile', 'account_number', 'aadhaar', 'name', 'dob', 'hindi_text']
            elif self.redaction_level == 'high':
                patterns_to_apply = list(self.patterns.keys())

            for entity in patterns_to_apply:
                matches = re.finditer(self.patterns[entity], page_text)
                for match in matches:
                    highlight_areas = page.search_for(match.group())
                    for area in highlight_areas:
                        page.add_redact_annot(area, fill=(0, 0, 0))
                        page.apply_redactions()

            for rect_coords in self.selected_rectangles[page_num]:
                x0, y0, x1, y1 = rect_coords
                manual_rect = fitz.Rect(x0, y0, x1, y1)
                page.add_redact_annot(manual_rect, fill=(0, 0, 0))
                page.apply_redactions()

        output_pdf = f"redacted_output_{self.redaction_level}.pdf"
        self.pdf_document.save(output_pdf)

        self.ask_for_password_protection(output_pdf)

    def ask_for_password_protection(self, pdf_path):
        response = messagebox.askyesno("Password Protection", "Do you want to set a password for the PDF?")
        if response:  # Agr koe user passwd select krta h tb 
            self.add_password_protection(pdf_path)
        else:
            messagebox.showinfo("PDF Saved", f"PDF saved without password as: {pdf_path}")

    def add_password_protection(self, pdf_path):
        password = simpledialog.askstring("Set Password", "Enter a password to protect the PDF:")

        if password:
            # Reading the redacted PDF
            reader = PdfReader(pdf_path)
            writer = PdfWriter()

            for page_num in range(len(reader.pages)):
                writer.add_page(reader.pages[page_num])

            writer.encrypt(password)

            # Saving the password
            protected_pdf_path = f"protected_{pdf_path}"
            with open(protected_pdf_path, "wb") as f_out:
                writer.write(f_out)

            messagebox.showinfo("Success", f"Password-protected PDF saved as: {protected_pdf_path}")
        else:
            messagebox.showwarning("No Password", "No password set. PDF saved without password protection.")

    def jump_to_page(self):
        page_number = simpledialog.askinteger("Jump to Page", "Enter page number:")
        if page_number and 1 <= page_number <= self.total_pages:
            self.current_page = page_number - 1
            self.display_pdf()
        else:
            messagebox.showerror("Invalid Page", "Please enter a valid page number.")

    def previous_page(self):
        if self.current_page > 0:
            self.current_page -= 1
            self.display_pdf()

    def next_page(self):
        if self.current_page < self.total_pages - 1:
            self.current_page += 1
            self.display_pdf()

    def start_rectangle(self, event):
        if self.manual_selection_enabled:
            self.rect_start = (event.x, event.y)
            self.rect = self.canvas.create_rectangle(self.rect_start[0], self.rect_start[1], event.x, event.y, outline="green")

    def update_rectangle(self, event):
        if self.manual_selection_enabled and self.rect:
            self.canvas.coords(self.rect, self.rect_start[0], self.rect_start[1], event.x, event.y)

    def end_rectangle(self, event):
        if self.manual_selection_enabled and self.rect:
            rect_coords = (self.rect_start[0], self.rect_start[1], event.x, event.y)
            self.selected_rectangles[self.current_page].append(rect_coords)
            self.canvas.delete(self.rect)

if __name__ == "__main__":
    PDFRedactor()